In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Dates
using HDF5

In [2]:
function bloch(ρ_list)
    # Pauli matricies
    σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]

    bloch_vec = [
        convert.(Float64, [ ρ[1,2]+ρ[2,1],
                           (ρ[1,2]-ρ[2,1])*im,
                            ρ[1,1]-ρ[2,2]       ])
                for ρ in ρ_list]
    hcat(bloch_vec...)
end

bloch (generic function with 1 method)

In [9]:
function exactDMDb4(γᵢ)

    evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    bᵉ = bloch(ρᵉ[1:lᵐᵃˣ])
    bᵍ = bloch(ρᵍ[1:lᵐᵃˣ])
    bˣ = bloch(ρˣ[1:lᵐᵃˣ])
    bʸ = bloch(ρʸ[1:lᵐᵃˣ])

    tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
    tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

    Δt = tᵉᶠ[2] - tᵉᶠ[1]

    Yᵉ = [bᵉ; ones(lᵐᵃˣ)']
    Yᵍ = [bᵍ; ones(lᵐᵃˣ)']
    Yˣ = [bˣ; ones(lᵐᵃˣ)']
    Yʸ = [bʸ; ones(lᵐᵃˣ)']

    Yᵉ⁻ = Yᵉ[:,1:end-1]; Yᵉ⁺ = Yᵉ[:,2:end]
    Yᵍ⁻ = Yᵍ[:,1:end-1]; Yᵍ⁺ = Yᵍ[:,2:end]
    Yˣ⁻ = Yˣ[:,1:end-1]; Yˣ⁺ = Yˣ[:,2:end]
    Yʸ⁻ = Yᵉ[:,1:end-1]; Yʸ⁺ = Yᵉ[:,2:end]

    Y⁻ = hcat(Yᵉ⁻, Yᵍ⁻, Yˣ⁻, Yʸ⁻)
    Y⁺ = hcat(Yᵉ⁺, Yᵍ⁺, Yˣ⁺, Yʸ⁺)

    A⁴ᴰ = Y⁺ * pinv(Y⁻) # Direct DMD estimation

    return A⁴ᴰ, Δt

end

exactDMDb4 (generic function with 1 method)

In [9]:
function exactDMDb4(γᵢ)
    data_dir = "../DATA/"

    γᵢ = string(γᵢ)

    if γᵢ == "0.0" 
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", "0.079477")
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", "0.079477")
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", "0.079477")
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", "0.079477")

        Hᴸᴹᴱ = [ 25.126        0
                 0             0    ]
        
        γᶠ = parse(Float64, γᵢ)

        Aᴸᴹᴱ = [ 0    √(γᶠ)
                 0     0. + 0im  ]

        tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
        tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

        ρᵉ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], tᵉᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρᵍ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], tᵍᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρˣ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], tˣᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρʸ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], tʸᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    else 
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    end

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    bᵉ = bloch(ρᵉ[1:lᵐᵃˣ])
    bᵍ = bloch(ρᵍ[1:lᵐᵃˣ])
    bˣ = bloch(ρˣ[1:lᵐᵃˣ])
    bʸ = bloch(ρʸ[1:lᵐᵃˣ])

    tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
    tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

    Δt = tᵉᶠ[2] - tᵉᶠ[1]

    Yᵉ = [bᵉ; ones(lᵐᵃˣ)']
    Yᵍ = [bᵍ; ones(lᵐᵃˣ)']
    Yˣ = [bˣ; ones(lᵐᵃˣ)']
    Yʸ = [bʸ; ones(lᵐᵃˣ)']

    Yᵉ⁻ = Yᵉ[:,1:end-1]; Yᵉ⁺ = Yᵉ[:,2:end]
    Yᵍ⁻ = Yᵍ[:,1:end-1]; Yᵍ⁺ = Yᵍ[:,2:end]
    Yˣ⁻ = Yˣ[:,1:end-1]; Yˣ⁺ = Yˣ[:,2:end]
    Yʸ⁻ = Yᵉ[:,1:end-1]; Yʸ⁺ = Yᵉ[:,2:end]

    Y⁻ = hcat(Yᵉ⁻, Yᵍ⁻, Yˣ⁻, Yʸ⁻)
    Y⁺ = hcat(Yᵉ⁺, Yᵍ⁺, Yˣ⁺, Yʸ⁺)

    A⁴ᴰ = Y⁺ * pinv(Y⁻) # Direct DMD estimation

    return A⁴ᴰ, Δt

end

exactDMDb4 (generic function with 1 method)

In [4]:
parentdir = ".."
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];
basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
test_files = dodeca_files;

../DATA/


In [10]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

models_data_file_name = "DMD_b4_LTI_trn4_"*date_and_time_string * ".h5"

println(models_data_file_name)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

#γ = ["0.0", "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

for γᵢ in  γ

    A⁴ᴰ, Δt = exactDMDb4(γᵢ)
    A⁴ᴰc = log(A⁴ᴰ)/Δt

    h5open(models_dir*models_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group
        γ_group["A"] = convert.(Float64, A⁴ᴰ)
        γ_group["Ac"] = convert.(Float64, A⁴ᴰc)
    end
end   

DMD_b4_LTI_trn4_2024-Sep-15_at_00-06.h5
